In [ ]:
# Setup and imports
import sys
sys.path.insert(0, '..')

import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
import yaml

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Load database credentials from secrets.yaml
with open('../config/secrets.yaml', 'r') as f:
    secrets = yaml.safe_load(f)

db_config = secrets.get('database', {})

# Build connection string
DB_HOST = db_config.get('host', 'localhost')
DB_PORT = db_config.get('port', 3306)
DB_USER = db_config.get('user', 'readonly')
DB_PASS = db_config.get('password', '')
DB_NAME = db_config.get('database', 'homeassistant')

connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
print(f"Connecting to: {DB_HOST}:{DB_PORT}/{DB_NAME} as {DB_USER}")

In [ ]:
# Create database connection
engine = create_engine(connection_string)

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT VERSION()"))
    row = result.fetchone()
    if row:
        version = row[0]
        print(f"✅ Connected to MariaDB: {version}")
    else:
        print("❌ Failed to get database version")

## 1. Explore Database Schema

In [ ]:
# List all tables
df_tables = pd.read_sql("SHOW TABLES", engine)
print("📋 Tables in homeassistant database:")
for table in df_tables.iloc[:, 0]:
    # Get row count for each table
    count = pd.read_sql(f"SELECT COUNT(*) as cnt FROM `{table}`", engine).iloc[0, 0]
    print(f"   • {table}: {count:,} rows")

## 2. States Data

The `states` table contains all entity state changes, linked to:
- `states_meta` - Entity ID mappings
- `state_attributes` - JSON attributes for each state

In [ ]:
# Preview states_meta (entity ID mappings)
df_entities = pd.read_sql("""
    SELECT metadata_id, entity_id 
    FROM states_meta 
    ORDER BY entity_id
""", engine)

print(f"Total entities tracked: {len(df_entities)}")
df_entities.head(20)

In [ ]:
# Define time range for extraction
days_back = 7  # Adjust as needed
end_time = datetime.now()
start_time = end_time - timedelta(days=days_back)

print(f"📅 Extraction range:")
print(f"   Start: {start_time}")
print(f"   End:   {end_time}")
print(f"   Days:  {days_back}")

In [ ]:
# Extract complete state history with entity IDs and attributes
query_states = """
SELECT 
    sm.entity_id,
    s.state,
    s.last_changed_ts,
    s.last_updated_ts,
    sa.shared_attrs
FROM states s
JOIN states_meta sm ON s.metadata_id = sm.metadata_id
LEFT JOIN state_attributes sa ON s.attributes_id = sa.attributes_id
WHERE s.last_updated_ts >= :start_ts
  AND s.last_updated_ts <= :end_ts
ORDER BY s.last_updated_ts
"""

# Execute query
df_states = pd.read_sql(
    text(query_states), 
    engine,
    params={
        'start_ts': start_time.timestamp(),
        'end_ts': end_time.timestamp()
    }
)

# Convert timestamps
df_states['last_changed'] = pd.to_datetime(df_states['last_changed_ts'], unit='s')
df_states['last_updated'] = pd.to_datetime(df_states['last_updated_ts'], unit='s')
df_states['domain'] = df_states['entity_id'].str.split('.').str[0]

print(f"📊 States extracted: {len(df_states):,} rows")
print(f"\nEvents by domain:")
print(df_states['domain'].value_counts().head(15))

In [ ]:
# Preview the data
df_states.head(10)

## 3. Long-Term Statistics

The `statistics` table contains hourly aggregated data for sensors that support it.

In [ ]:
# Get statistics metadata (which entities have statistics)
df_stats_meta = pd.read_sql("""
    SELECT id, statistic_id, source, unit_of_measurement
    FROM statistics_meta
    ORDER BY statistic_id
""", engine)

print(f"Entities with long-term statistics: {len(df_stats_meta)}")
df_stats_meta.head(20)

In [ ]:
# Extract long-term statistics (hourly)
query_statistics = """
SELECT 
    sm.statistic_id,
    sm.unit_of_measurement,
    s.start_ts,
    s.mean,
    s.min,
    s.max,
    s.sum,
    s.state
FROM statistics s
JOIN statistics_meta sm ON s.metadata_id = sm.id
WHERE s.start_ts >= :start_ts
  AND s.start_ts <= :end_ts
ORDER BY s.start_ts
"""

df_statistics = pd.read_sql(
    text(query_statistics),
    engine,
    params={
        'start_ts': start_time.timestamp(),
        'end_ts': end_time.timestamp()
    }
)

# Convert timestamps
df_statistics['timestamp'] = pd.to_datetime(df_statistics['start_ts'], unit='s')

print(f"📊 Statistics extracted: {len(df_statistics):,} rows")
df_statistics.head(10)

In [ ]:
# Statistics by entity
print("📊 Statistics records by entity:")
print(df_statistics['statistic_id'].value_counts().head(20))

## 4. Short-Term Statistics (5-minute resolution)

In [ ]:
# Extract short-term statistics (5-minute intervals)
query_short_term = """
SELECT 
    sm.statistic_id,
    sm.unit_of_measurement,
    s.start_ts,
    s.mean,
    s.min,
    s.max,
    s.sum,
    s.state
FROM statistics_short_term s
JOIN statistics_meta sm ON s.metadata_id = sm.id
WHERE s.start_ts >= :start_ts
  AND s.start_ts <= :end_ts
ORDER BY s.start_ts
"""

df_short_term = pd.read_sql(
    text(query_short_term),
    engine,
    params={
        'start_ts': start_time.timestamp(),
        'end_ts': end_time.timestamp()
    }
)

# Convert timestamps
df_short_term['timestamp'] = pd.to_datetime(df_short_term['start_ts'], unit='s')

print(f"📊 Short-term statistics: {len(df_short_term):,} rows")
df_short_term.head(10)

## 5. Events Data

In [ ]:
# Get event types
df_event_types = pd.read_sql("""
    SELECT event_type_id, event_type
    FROM event_types
    ORDER BY event_type
""", engine)

print(f"Event types tracked: {len(df_event_types)}")
df_event_types

In [ ]:
# Extract events (can be large, so limiting)
query_events = """
SELECT 
    et.event_type,
    e.time_fired_ts,
    ed.shared_data
FROM events e
JOIN event_types et ON e.event_type_id = et.event_type_id
LEFT JOIN event_data ed ON e.data_id = ed.data_id
WHERE e.time_fired_ts >= :start_ts
  AND e.time_fired_ts <= :end_ts
ORDER BY e.time_fired_ts
LIMIT 100000
"""

df_events = pd.read_sql(
    text(query_events),
    engine,
    params={
        'start_ts': start_time.timestamp(),
        'end_ts': end_time.timestamp()
    }
)

# Convert timestamps
df_events['time_fired'] = pd.to_datetime(df_events['time_fired_ts'], unit='s')

print(f"📊 Events extracted: {len(df_events):,} rows")
print(f"\nEvents by type:")
print(df_events['event_type'].value_counts().head(15))

## 6. Export Data

In [ ]:
# Setup export directory
from pathlib import Path
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
print(f"📁 Export directory: {data_dir.resolve()}")
print(f"📅 Timestamp: {timestamp}")

In [ ]:
# Export all data to CSV
exports = [
    ('states', df_states),
    ('statistics', df_statistics),
    ('statistics_short_term', df_short_term),
    ('events', df_events),
]

# Track successful exports for cleanup
exported_files = []

for name, df in exports:
    if not df.empty:
        filepath = data_dir / f'{name}_{timestamp}.csv'
        df.to_csv(filepath, index=False)
        exported_files.append((name, filepath))
        print(f"✅ {name}: {len(df):,} rows → {filepath.name}")
    else:
        print(f"⚠️ {name}: No data to export")

# Clean up older exports if all current exports succeeded
if len(exported_files) == len([e for e in exports if not e[1].empty]):
    removed_count = 0
    for name, _ in exported_files:
        # Find older files with same prefix
        pattern = f'{name}_*.csv'
        for old_file in data_dir.glob(pattern):
            # Skip current export
            if f'_{timestamp}.csv' not in old_file.name:
                old_file.unlink()
                removed_count += 1
    if removed_count > 0:
        print(f"\n🧹 Cleaned up {removed_count} older export file(s)")

print("\n📊 Export complete!")

## 7. Full Historical Export (Optional)

For complete historical data without time limits.

In [ ]:
# Uncomment to export ALL historical data (can be very large!)

# Full states export
# df_all_states = pd.read_sql("""
#     SELECT 
#         sm.entity_id,
#         s.state,
#         s.last_changed_ts,
#         s.last_updated_ts,
#         sa.shared_attrs
#     FROM states s
#     JOIN states_meta sm ON s.metadata_id = sm.metadata_id
#     LEFT JOIN state_attributes sa ON s.attributes_id = sa.attributes_id
#     ORDER BY s.last_updated_ts
# """, engine)
# 
# df_all_states.to_csv(data_dir / f'all_states_{timestamp}.csv', index=False)
# print(f"Exported {len(df_all_states):,} total state records")

In [ ]:
# Close connection
engine.dispose()
print("✅ Database connection closed")

## Summary

This notebook extracted directly from MariaDB:
1. **States** - Complete entity state history with attributes
2. **Statistics** - Hourly aggregated data for supported sensors
3. **Short-term Statistics** - 5-minute resolution data
4. **Events** - System events with data

### Advantages over REST API:
- ⚡ Much faster (single query vs many API calls)
- 📊 No time range limits
- 🔍 Full access to raw data and attributes
- 📈 Direct access to statistics tables